In [ ]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Categories=['yes','no']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='/content/drive/MyDrive/PCOSrevised'
#path which contains all the categories of images
for i in Categories:

	print(f'loading... category : {i}')
	path=os.path.join(datadir,i)
	for img in os.listdir(path):
		img_array=imread(os.path.join(path,img))
		img_resized=resize(img_array,(150,150,3))
		flat_data_arr.append(img_resized.flatten())
		target_arr.append(Categories.index(i))
	print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)


loading... category : yes
loaded category:yes successfully
loading... category : no
loaded category:no successfully


In [ ]:
print(flat_data.shape)
print(flat_data)

(125, 67500)
[[0.11722633 0.11193385 0.10492441 ... 0.1013014  0.06250798 0.03680768]
 [0.34493007 0.24769366 0.20167455 ... 0.08538071 0.08930228 0.10229214]
 [0.29727222 0.30714042 0.30321301 ... 0.19063478 0.12788966 0.07385378]
 ...
 [0.57763195 0.46816741 0.39530577 ... 0.91758801 0.75555285 0.67114411]
 [0.24791087 0.17732664 0.16162834 ... 0.1531385  0.11390684 0.11704503]
 [0.23251427 0.16192604 0.14623976 ... 0.16992348 0.13029042 0.12244274]]


In [ ]:
#dataframe
df=pd.DataFrame(flat_data)
df['Target']=target
df.shape

(125, 67501)

In [ ]:
#input data
x=df.iloc[:,:-1]
#output data
y=df.iloc[:,-1]


In [ ]:
# Splitting the data into training and testing sets
x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.20,
											random_state=77,
											stratify=y)


In [ ]:
# Defining the parameters grid for GridSearchCV
param_grid={'C':[0.1,1,10,100],
			'gamma':[0.0001,0.001,0.1,1],
			'kernel':['rbf','poly']}

# Creating a support vector classifier
svc=svm.SVC(probability=True)

# Creating a model using GridSearchCV with the parameters grid
model=GridSearchCV(svc,param_grid)


In [ ]:
# Training the model using the training data
model.fit(x_train,y_train)


GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.0001, 0.001, 0.1, 1],
                         'kernel': ['rbf', 'poly']})

In [ ]:
# Testing the model using the testing data
y_pred = model.predict(x_test)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_pred, y_test)

# Print the accuracy of the model
print(f"The model is {accuracy*100}% accurate")


The model is 96.0% accurate


In [ ]:
print(classification_report(y_test, y_pred, target_names=['yes', 'no']))


              precision    recall  f1-score   support

         yes       1.00      0.91      0.95        11
          no       0.93      1.00      0.97        14

    accuracy                           0.96        25
   macro avg       0.97      0.95      0.96        25
weighted avg       0.96      0.96      0.96        25



In [ ]:
import pickle

# Assuming 'model' is your trained model object
# Save the model to a file named 'svm_model.pkl'
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
# Load the saved model from the file
with open('svm_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
